## Import Needed Libraries and Filepaths

In [13]:
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from texas_gerrymandering_hb4.config import IMAGES_DIR

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

ART_DIR = Path("artifacts")
ART_DIR.mkdir(parents=True, exist_ok=True)

## Load Data and Metadata

In [14]:
X_train = pd.read_parquet(ART_DIR / "X_train.parquet")
y_train = pd.read_parquet(ART_DIR / "y_train.parquet")["party"]

with open(ART_DIR / "split_meta.json") as f:
    meta = json.load(f)
NUMERIC = meta["numeric"]
CATEGORICAL = meta["categorical"]

In [15]:
# --- Cell 3: Preprocessor ---
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(drop="first", sparse_output=False), CATEGORICAL),
    ("num", StandardScaler(), NUMERIC),
])

# Helper to recover feature names after fit
def get_feature_names(fitted_preprocessor):
    ohe = fitted_preprocessor.named_transformers_["cat"]
    ohe_names = list(ohe.get_feature_names_out(CATEGORICAL))
    feat_names = ohe_names + NUMERIC
    return ohe_names, feat_names

## Tuning Threshold

In [16]:
def pick_threshold(y_true, scores, metric="balanced_accuracy"):
    scores = np.asarray(scores)
    grid = np.linspace(0.0, 1.0, 201)
    best_thr, best_val = 0.5, -1.0
    for thr in grid:
        y_hat = (scores >= thr).astype(int)
        val = balanced_accuracy_score(y_true, y_hat) if metric == "balanced_accuracy" else 0.0
        if val > best_val:
            best_val, best_thr = val, thr
    return float(best_thr), float(best_val)

## K-Fold Cross Validation

In [17]:
# --- Cell 5: Cross-val evaluator for regression-as-classifier ---
def cv_bal_acc_for_reg_pipeline(pipeline, X, y, n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    scores = []
    for tr_idx, va_idx in kf.split(X):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]
        # fit on fold-train
        pipeline.fit(X_tr, y_tr)
        # tune threshold on fold-train ONLY
        y_tr_score = pipeline.predict(X_tr).clip(0, 1)
        thr, _ = pick_threshold(y_tr, y_tr_score, "balanced_accuracy")
        # evaluate on fold-val
        y_va_score = pipeline.predict(X_va).clip(0, 1)
        y_va_pred = (y_va_score >= thr).astype(int)
        bal = balanced_accuracy_score(y_va, y_va_pred)
        scores.append(bal)
    return float(np.mean(scores)), float(np.std(scores))

In [18]:
# --- Cell 6: Define model candidates ---
alphas = np.logspace(-3, 3, 50)
candidates = {
    "ols": LinearRegression(),
    "ridge": RidgeCV(alphas=alphas, cv=5, scoring=None),
    "lasso": LassoCV(alphas=alphas, cv=5, max_iter=10000, n_jobs=-1),
    "elasticnet": ElasticNetCV(l1_ratio=[0.2, 0.5, 0.8, 1.0], alphas=alphas, cv=5, max_iter=10000, n_jobs=-1)
}

results = {}
best_name, best_cv = None, -np.inf

for name, reg in candidates.items():
    pipe = Pipeline([("pre", preprocessor), ("reg", reg)])
    mean_bal, std_bal = cv_bal_acc_for_reg_pipeline(pipe, X_train, y_train, n_splits=5, random_state=42)
    results[name] = {"cv_balanced_accuracy_mean": mean_bal, "cv_balanced_accuracy_std": std_bal}
    print(f"[CV] {name}: mean={mean_bal:.3f} ± {std_bal:.3f}")
    if mean_bal > best_cv:
        best_cv = mean_bal
        best_name = name
best_reg = candidates[best_name]
print(f"[SELECT] Best by CV balanced accuracy: {best_name} (mean={best_cv:.3f})")

/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[CV] ols: mean=0.825 ± 0.113


/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:2801: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


[CV] ridge: mean=0.850 ± 0.133


/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


[CV] lasso: mean=0.883 ± 0.163


/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_classification.py:534: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


[CV] elasticnet: mean=0.883 ± 0.163
[SELECT] Best by CV balanced accuracy: lasso (mean=0.883)


In [19]:
# --- Cell 7: Fit best on full train & tune final threshold ---
best_pipeline = Pipeline([("pre", preprocessor), ("reg", best_reg)])
best_pipeline.fit(X_train, y_train)
y_train_score = best_pipeline.predict(X_train).clip(0, 1)
final_thr, bal_train = pick_threshold(y_train, y_train_score, "balanced_accuracy")
print(f"[TRAIN] Final threshold={final_thr:.3f}, balanced-acc(train)={bal_train:.3f}")

[TRAIN] Final threshold=0.200, balanced-acc(train)=0.932


In [20]:
# --- Cell 8: Save model, threshold, CV results, coefficients plot ---
joblib.dump(best_pipeline, ART_DIR / "active_model.pkl")
with open(ART_DIR / "train_threshold.json", "w") as f:
    json.dump(
        {"variant": best_name, "threshold": final_thr, "balanced_accuracy_on_train": bal_train},
        f, indent=2
    )
with open(ART_DIR / "cv_results.json", "w") as f:
    json.dump(results, f, indent=2)

# coefficients
ohe_names, feat_names = get_feature_names(best_pipeline.named_steps["pre"])
coefs = best_pipeline.named_steps["reg"].coef_
coef_df = pd.DataFrame({"feature": feat_names, "coef": coefs}).sort_values(
    "coef", key=lambda s: s.abs(), ascending=False
)
coef_df.to_csv(ART_DIR / "linear_regression_coefficients_active.csv", index=False)

plt.figure()
plt.barh(coef_df["feature"], coef_df["coef"])
plt.title(f"Linear Regression Coefficients ({best_name})")
plt.xlabel("Coefficient Value")
plt.tight_layout()
plt.savefig(IMAGES_DIR / "linear_regression_coefficients_active.png", dpi=200)
plt.close()

## Model Diagnostics

In [21]:
# --- Cell 9: Baseline OLS diagnostics on train (VIF, residuals, Cook's D) ---
# Fit a separate OLS baseline for diagnostics only
ols_pipe = Pipeline([("pre", preprocessor), ("reg", LinearRegression())])
ols_pipe.fit(X_train, y_train)
X_design = ols_pipe.named_steps["pre"].transform(X_train)  # dense array
ohe_names_diag = list(ols_pipe.named_steps["pre"].named_transformers_["cat"].get_feature_names_out(CATEGORICAL))
feat_names_diag = ohe_names_diag + NUMERIC

# statsmodels OLS
X_sm = sm.add_constant(X_design)
ols_sm = sm.OLS(y_train.values, X_sm).fit()

# VIF
vif_rows = []
for i in range(1, X_sm.shape[1]):  # skip intercept
    vif_rows.append((feat_names_diag[i-1], float(variance_inflation_factor(X_sm[:, 1:], i-1))))
vif_df = pd.DataFrame(vif_rows, columns=["feature", "VIF"]).sort_values("VIF", ascending=False)
vif_df.to_csv(ART_DIR / "vif_ols.csv", index=False)

plt.figure()
plt.barh(vif_df["feature"], vif_df["VIF"])
plt.title("VIF (OLS baseline, Train)")
plt.xlabel("VIF")
plt.tight_layout()
plt.savefig(IMAGES_DIR / "vif_ols.png", dpi=200)
plt.close()

# residuals & QQ
y_fit = ols_pipe.predict(X_train).clip(0, 1)
resid = y_train.values - y_fit

plt.figure(); plt.scatter(y_fit, resid); plt.axhline(0, ls="--")
plt.title("Residuals vs Fitted (OLS baseline)")
plt.xlabel("Fitted"); plt.ylabel("Residual"); plt.tight_layout()
plt.savefig(IMAGES_DIR / "residuals_vs_fitted_ols.png", dpi=200); plt.close()

plt.figure(); plt.hist(resid, bins=12); plt.title("Residuals Histogram (OLS baseline)")
plt.tight_layout(); plt.savefig(IMAGES_DIR / "residuals_hist_ols.png", dpi=200); plt.close()

fig = sm.qqplot(resid, line='45', fit=True); plt.title("QQ Plot (OLS baseline)")
plt.tight_layout(); fig.savefig(IMAGES_DIR / "qqplot_residuals_ols.png", dpi=200); plt.close()

# Influence / Cook's D
influence = ols_sm.get_influence()
cooks_d = influence.cooks_distance[0]
leverage = influence.hat_matrix_diag
infl_df = pd.DataFrame({"index": X_train.index, "cooks_d": cooks_d, "leverage": leverage, "residual": resid}).set_index("index")
infl_df.sort_values("cooks_d", ascending=False).to_csv(ART_DIR / "influence_ols.csv")

plt.figure(); plt.scatter(leverage, resid)
plt.title("Leverage vs Residuals (OLS baseline)")
plt.xlabel("Leverage"); plt.ylabel("Residual"); plt.tight_layout()
plt.savefig(IMAGES_DIR / "leverage_vs_residuals_ols.png", dpi=200); plt.close()


In [22]:
# --- Cell 10: Wrap up ---
with open(ART_DIR / "training_summary.json", "w") as f:
    json.dump({
        "selected_variant": best_name,
        "train_balanced_accuracy": bal_train,
        "final_threshold": final_thr,
        "cv_results": results
    }, f, indent=2)

print("Training complete. Saved active_model.pkl, threshold, CV results, coefficients, and diagnostics.")

Training complete. Saved active_model.pkl, threshold, CV results, coefficients, and diagnostics.
